## Image Visualization

In [1]:
from rikai.types.vision import Image
uri = 'https://i.scdn.co/image/ab67616d0000b273466def3ce70d94dcacb13c8d'
image = Image(uri)
image

Image(uri=https://i.scdn.co/image/ab67616d0000b273466def3ce70d94dcacb13c8d)

## Initialization
+ Initialize the Spark Session with Rikai support
+ Register all built-in UDFs
+ Create the yolov5m model

In [2]:
from rikai.spark.utils import init_spark_session
from rikai.spark.functions import init

spark = init_spark_session(
    dict(
        [
            (
                "spark.rikai.sql.ml.registry.torchhub.impl",
                "ai.eto.rikai.sql.model.torchhub.TorchHubRegistry",
            )
        ]
    )
)

init(spark)

:: loading settings :: url = jar:file:/Users/da/.pyenv/versions/3.8.10/envs/rikai-yolov5/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/da/.ivy2/cache
The jars for the packages stored in: /Users/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f34c09f4-88bf-4a35-8fd2-1edf6a75eb93;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.0.20 in central
	found org.antlr#antlr4-runtime;4.8-1 in local-m2-cache
	found com.thoughtworks.enableIf#enableif_2.12;1.1.7 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found io.circe#circe-core_2.12;0.12.3 in central
	found io.circe#circe-numbers_2.12;0.12.3 in central
	found org.typelevel#cats-core_2.12;2.0.0 in central
	found org.typelevel#cats-macros_2.12;2.0.0 in central
	found org.typelevel#cats-kernel_2.12;2.0.0 in central
	found io.circe#circe-generic_2.12;0.12.3 in central
	found com.chuusai#shapeless_2.12;2.3.3 in spark-list
	found org.typeleve

In [3]:
spark.sql("""
CREATE MODEL yolov5m
FLAVOR yolov5 
OPTIONS (device="cpu", batch_size=32)
USING "torchhub:///ultralytics/yolov5:v6.0/yolov5m";
""")

DataFrame[]

In [4]:
spark.sql(f"""
select pred.box, pred.label, pred.score
from (
  select explode(ML_PREDICT(yolov5m, to_image('{uri}'))) as pred
)
""").toPandas()

Using cache found in /Users/da/.cache/torch/hub/ultralytics_yolov5_v6.0+ 1) / 1]
YOLOv5 🚀 2021-12-29 torch 1.10.1 CPU

Fusing layers... 
/Users/da/.pyenv/versions/rikai-yolov5/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 
/Users/da/.pyenv/versions/rikai-yolov5/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Using cache found in /Users/da/.cache/torch/hub/ultralytics_yolov5_v6.0
YOLOv5 🚀 2021-12-29 torch 1.10.1 CPU

Fusing layers... 
/Users/da/.pyenv/versions/rikai-yolov

,box,label,score
0,"(284.51934814453125, 151.3018341064453, 440.52...",person,0.893445
1,"(260.7940673828125, 184.31243896484375, 363.49...",person,0.882638
2,"(168.14048767089844, 310.9768371582031, 199.94...",car,0.760498
3,"(117.99897003173828, 314.7746887207031, 131.33...",car,0.725537
4,"(79.77767944335938, 304.52392578125, 105.77915...",truck,0.430030
5,"(101.65540313720703, 316.5750732421875, 112.83...",car,0.347411
6,"(215.78570556640625, 303.4993591308594, 224.52...",person,0.342589
7,"(233.14926147460938, 176.81866455078125, 635.6...",truck,0.323248
8,"(79.44734191894531, 305.41943359375, 105.02845...",car,0.290373
9,"(160.8476104736328, 309.04046630859375, 171.00...",person,0.286691


## Q: How many cars/persons in this picture

In [5]:
df = spark.sql(f"""
select pred.label, count(*)
from (
  select *, explode(ML_PREDICT(yolov5m, image)) as pred
  from (
    select to_image('{uri}') as image
  )
)
group by pred.label
""")
df.toPandas()

Using cache found in /Users/da/.cache/torch/hub/ultralytics_yolov5_v6.0
YOLOv5 🚀 2021-12-29 torch 1.10.1 CPU

Fusing layers... 
Model Summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 
/Users/da/.pyenv/versions/rikai-yolov5/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Using cache found in /Users/da/.cache/torch/hub/ultralytics_yolov5_v6.0
YOLOv5 🚀 2021-12-29 torch 1.10.1 CPU

Fusing layers... 
/Users/da/.pyenv/versions/rikai-yolov5/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 290 layers, 21172173 parameters

,label,count(1)
0,person,5
1,truck,2
2,car,4


## Q2: Where are these cars and persons?

In [6]:
from rikai.viz import Text
from rikai.types.vision import Image
from functools import reduce

df = spark.sql(f"""
select * from (
  select explode(ML_PREDICT(yolov5m, image)) as pred
  from (
    select to_image('{uri}') as image
  )
)
where pred.label = 'truck' -- truck/car/person
""")
preds = [row.pred for row in df.collect()]

def add_layer(image, pred):
    return image | pred.box | Text(pred.label, (pred.box.xmin, pred.box.ymin - 10))

reduce(lambda a,b: add_layer(a, b), [image] + preds)

Using cache found in /Users/da/.cache/torch/hub/ultralytics_yolov5_v6.0
YOLOv5 🚀 2021-12-29 torch 1.10.1 CPU

Fusing layers... 
Model Summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 
/Users/da/.pyenv/versions/rikai-yolov5/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Using cache found in /Users/da/.cache/torch/hub/ultralytics_yolov5_v6.0
YOLOv5 🚀 2021-12-29 torch 1.10.1 CPU

Fusing layers... 
Model Summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 
/Users/da/.pyenv/versions/rikai-yolov5/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Draw(Box2d(xmin=79.77767944335938, ymin=304.52392578125, xmax=105.77915954589844, ymax=332.91534423828125))